### 第1セル

In [1]:
# RAGデータの設定
DATA_FILE_PATH = "大谷翔平_raw.txt"  # この変数を変更してデータファイルを指定

### 第2セル

In [2]:
# ファインチューニング設定
DEFAULT_LORA_ADAPTER_PATH = "./checkpoint-160"  # チェックポイントのパス（例: "./chackpoint-100"）

### 第3セル

In [ ]:
import gradio as gr
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM, BitsAndBytesConfig
import faiss
import re
import unicodedata
from typing import List, Tuple
import os
from pathlib import Path
import time  # 時間計測用
import pykakasi
import socket

# OpenMPの問題を解決するための環境変数設定
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

# ==============
# グローバル設定（外部で変更可能）
# ==============
# 使用するデータファイルのパスをここで設定
DATA_FILE_PATH = "大谷翔平_raw.txt"  # この変数を変更してデータファイルを指定

# デフォルトモデルパスの設定
DEFAULT_LLM_MODEL_NAME = "./sbintuitions/sarashina2.2-3B-instruct-v0.1"
DEFAULT_EMB_MODEL_NAME = "./bge-m3"
DEFAULT_LORA_ADAPTER_PATH = "./output/checkpoint-160"  # LoRAアダプターのフォルダ名を指定（例: "./output/checkpoint-160" または "./my-lora-adapter"）

class ChangeLLMChatbot:
    def __init__(self):
        # データファイルパス（外部で設定可能）
        self.text_path = DATA_FILE_PATH
        
        # デフォルトモデルパスの設定
        self.llm_model_name = DEFAULT_LLM_MODEL_NAME
        self.emb_model_name = DEFAULT_EMB_MODEL_NAME
        self.lora_adapter_path = DEFAULT_LORA_ADAPTER_PATH
        
        # モデルタイプ : original or finetuning
        self.model_type = "original"
        
        # 生成設定
        self.temperature = 0.7
        self.answer_token_limit = 200
        self.answer_headroom = 50
        self.top_k_retrieve = 3
        self.gen_mode = "RAG"  # "RAG" or "FT"
        
        # メモリ最適化設定
        self.chunk_size = 100
        self.batch_size = 4
        
        # RAG設定
        self.rag_user_prompt_template = ""
        self.use_chat_template = True
        
        # ファインチューニング設定
        self.ft_system_prompt = ""
        
        # 実行環境
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 設定インスタンスを作成
cfg = ChangeLLMChatbot()



# グローバル変数でモデルを保持
llm_tok = None
llm = None  # ベースモデル（常に保持）
llm_with_lora = None  # LoRA適用済みモデル（一度だけ作成）
emb_tok = None
emb_model = None

# FAISSインデックスフォルダの設定
FAISS_INDEX_DIR = "faiss_index"

# FAISSインデックスフォルダが存在しない場合は作成
if not os.path.exists(FAISS_INDEX_DIR):
    os.makedirs(FAISS_INDEX_DIR)

# FAISSインデックスとチャンクデータを保持
faiss_index = None
chunk_data = None
last_data_file = None  # 最後に読み込んだデータファイルを記録

def find_available_port(start_port=7860, max_attempts=100):
    """利用可能なポートを見つける"""
    for port in range(start_port, start_port + max_attempts):
        try:
            with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
                s.bind(('0.0.0.0', port))
                return port
        except OSError:
            continue
    return None

# ==============
# RAGデータ管理
# ==============
def convert_to_romaji(filename):
    """日本語ファイル名をローマ字に変換"""
    try:
        # pykakasiの新しいAPIを使用
        kakasi = pykakasi.kakasi()
        
        # 拡張子を除去
        name_without_ext = Path(filename).stem
        
        # ローマ字変換
        romaji_result = kakasi.convert(name_without_ext)
        
        # pykakasiの結果がリストの場合は、hepburn形式の文字列を結合
        if isinstance(romaji_result, list):
            romaji_name = ""
            for item in romaji_result:
                if isinstance(item, dict) and 'hepburn' in item:
                    romaji_name += item['hepburn']
                elif isinstance(item, str):
                    romaji_name += item
        else:
            romaji_name = str(romaji_result)
        
        # 英数字以外の文字を除去し、スペースをアンダースコアに変換
        safe_name = re.sub(r'[^a-zA-Z0-9]', '_', romaji_name)
        safe_name = re.sub(r'_+', '_', safe_name)  # 連続するアンダースコアを1つに
        safe_name = safe_name.strip('_')  # 前後のアンダースコアを除去
        
        return safe_name if safe_name else "data"
        
    except Exception as e:
        return "data"

def get_faiss_index_path(data_filename):
    """FAISSインデックスファイルのパスを取得"""
    # 日本語判定（ひらがな、カタカナ、漢字が含まれているか）
    japanese_pattern = re.compile(r'[\u3040-\u309F\u30A0-\u30FF\u4E00-\u9FAF]')
    
    if japanese_pattern.search(data_filename):
        # 日本語の場合、ローマ字変換
        romaji_name = convert_to_romaji(data_filename)
        return os.path.join(FAISS_INDEX_DIR, f"{romaji_name}.faiss")
    else:
        # 日本語でない場合、そのまま使用
        safe_name = re.sub(r'[^a-zA-Z0-9._-]', '_', data_filename)
        return os.path.join(FAISS_INDEX_DIR, f"{safe_name}.faiss")

def update_rag_data():
    """RAGデータを更新（FAISSインデックスファイルによる条件分岐）"""
    global faiss_index, chunk_data, last_data_file
    
    current_file = cfg.text_path
    current_filename = Path(current_file).name  # ファイル名（拡張子含む）
    
    # FAISSインデックスファイルのパスを取得
    faiss_index_path = get_faiss_index_path(current_filename)
    
    # 既存のインデックスファイルが存在する場合
    if os.path.exists(faiss_index_path):
        try:
            # インデックスファイルから読み込み
            faiss_index = faiss.read_index(faiss_index_path)
            # チャンクデータも再構築
            text = Path(current_file).read_text(encoding="utf-8")
            chunk_data = create_chunks(text)
            last_data_file = current_file
            return True
        except Exception as e:
            pass
    
    # 新しいデータを読み込み、インデックスを構築
    if current_file != last_data_file or faiss_index is None:
        try:
            text = Path(current_file).read_text(encoding="utf-8")
            chunk_data = create_chunks(text)
            
            # 新しいインデックスを構築
            chunk_embs = embed_texts(chunk_data, emb_tok, emb_model)
            faiss_index = build_faiss_index(chunk_embs)
            
            # インデックスファイルを保存
            faiss.write_index(faiss_index, faiss_index_path)
            
            last_data_file = current_file
            
        except Exception as e:
            return False
    
    return True

# ==============
# 文字正規化
# ==============
def _normalize(text: str) -> str:
    # 改行削除＋NFKC＋lower
    return unicodedata.normalize("NFKC", text.replace("\n", "")).lower()

def create_chunks(text: str, chunk_size: int = None, overlap: bool = False, overlap_size: int = 20) -> List[str]:
    #  改行削除＋NFKC＋lower
    text = _normalize(text)
    
    if not text:
        return []
    
    # 設定値を使用
    if chunk_size is None:
        chunk_size = cfg.chunk_size
    
    chunks: List[str] = []
    step = max(1, chunk_size - overlap_size) if overlap else chunk_size
    for i in range(0, len(text), step):
        piece = text[i:i + chunk_size]
        if piece:
            chunks.append(piece)
    return chunks
    
def load_generation_model():
    """生成モデルを読み込み（4bit量子化対応）"""
    global llm_tok, llm
    
    try:
        # 常にベースモデル（sarashina）を読み込み
        model_name = cfg.llm_model_name
        print(f"モデルパス: {model_name}")
        
        # 4bit量子化設定（メモリ最適化）
        print("4bit量子化設定を作成中...")
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,  # 二重量子化を有効化（AI_chatbot.pyと同じ）
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16
        )
        
        # トークナイザーを読み込み
        print("トークナイザーを読み込み中...")
        global llm_tok
        llm_tok = AutoTokenizer.from_pretrained(model_name)
        print("トークナイザーの読み込み完了")
        
        # モデルを4bit量子化で読み込み（LoRAアダプターを無視）
        print("4bit量子化モデルを読み込み中...")
        global llm
        llm = AutoModelForCausalLM.from_pretrained(
            model_name,
            quantization_config=bnb_config,
            device_map="auto",
            torch_dtype=torch.float16,  # AI_chatbot.pyと同じ
            trust_remote_code=True,
            low_cpu_mem_usage=True,
            # LoRAアダプターを無視してベースモデルのみを読み込み
            ignore_mismatched_sizes=True
        )
        
        # pad_tokenが設定されていない場合の処理
        if llm_tok.pad_token_id is None and llm_tok.eos_token_id is not None:
            llm_tok.pad_token = llm_tok.eos_token
        llm.config.pad_token_id = llm_tok.pad_token_id or llm_tok.eos_token_id
        
        # vocab と lm_head の出力次元を一致＋可能ならtie
        vocab_size = llm_tok.vocab_size
        head = getattr(llm, "lm_head", None)
        head_out = getattr(head, "out_features", None) if head is not None else None
        if head_out is not None and head_out != vocab_size:
            print(f"[WARN] lm_head({head_out}) != vocab({vocab_size}) → resize")
            llm.resize_token_embeddings(vocab_size)
        try:
            llm.tie_weights()
        except Exception:
            pass
        try:
            with torch.no_grad():
                if hasattr(llm, "lm_head") and hasattr(llm, "model") and hasattr(llm.model, "embed_tokens"):
                    llm.lm_head.weight = llm.model.embed_tokens.weight
        except Exception:
            pass
        
        print("ベースモデル（sarashina）の読み込み完了")
        
        # 推論モードに設定
        llm.eval()
        
        return llm_tok, llm
        
    except Exception as e:
        print(f"モデルの読み込みエラー: {e}")
        return None, None

def apply_lora_adapter(base_model, adapter_path):
    """LoRAアダプターを適用（一度だけ実行）"""
    try:
        # パスの存在確認
        if not os.path.exists(adapter_path):
            print(f"LoRAアダプターパスが存在しません: {adapter_path}")
            return None
        
        from peft import PeftModel, PeftConfig
        import warnings
        
        print(f"LoRAアダプターを適用中: {adapter_path}")
        
        # LoRA設定の読み込み
        peft_config = PeftConfig.from_pretrained(adapter_path)
        print(f"LoRA設定を読み込みました: {peft_config.base_model_name_or_path}")
        
        # 警告を一時的に抑制
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=UserWarning, module="peft")
            
            # アダプターの適用
            lora_model = PeftModel.from_pretrained(
                base_model,
                adapter_path,
                is_trainable=False,  # 推論時はFalse
                torch_dtype=torch.float32
            )
        
        print("LoRAアダプターの適用完了")
        return lora_model
        
    except ImportError as e:
        print(f"PEFTライブラリがインストールされていません: {e}")
        return None
    except Exception as e:
        print(f"LoRAアダプターの適用に失敗: {e}")
        return None
        
# =========
# プロンプト（外出し対応）
# =========
DEFAULT_RAG_USER = (
    "与えられた情報だけで日本語で簡潔かつ正確に回答してください。\n"
    "与えられた情報にないことは推測で書かず、わからない場合はわからないと言ってください。\n\n"
    "[質問]\n{question}\n\n"
    "[情報]\n{context}\n\n"
    "[回答]\n"
)

DEFAULT_FT_PROMPT = "日本語で簡潔かつ正確に回答してください。根拠が無い推測は避けてください。"

def _safe_format(template: str, **kwargs) -> str:
    class _D(dict):
        def __missing__(self, k): return "{"+k+"}"
    return template.format_map(_D(**kwargs))

@torch.no_grad()
def build_prompt_rag(cfg, question: str, retrieved_chunks: List[str], tok: AutoTokenizer) -> str:
    context = "\n---\n".join(retrieved_chunks)
    user_tmpl = cfg.rag_user_prompt_template.strip() if cfg.rag_user_prompt_template else DEFAULT_RAG_USER
    user_content = _safe_format(user_tmpl, question=question, context=context)

    if cfg.use_chat_template and hasattr(tok, "apply_chat_template"):
        messages = [
            {"role": "user", "content": user_content},
        ]
        return tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return user_content

@torch.no_grad()
def build_prompt_ft(cfg, question: str, tok: AutoTokenizer) -> str:
    system_tmpl = cfg.ft_system_prompt.strip() if cfg.ft_system_prompt else DEFAULT_FT_PROMPT
    
    if cfg.use_chat_template and hasattr(tok, "apply_chat_template"):
        messages = [
            {"role": "system", "content": system_tmpl},
            {"role": "user", "content": question},
        ]
        return tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return question
        



def mean_pooling(model_output, attention_mask):
    """平均プーリングによるベクトル化"""
    token_embeddings = model_output.last_hidden_state
    mask = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    summed = (token_embeddings * mask).sum(dim=1)
    counts = mask.sum(dim=1).clamp(min=1e-9)
    return summed / counts

def embed_texts(texts: List[str], emb_tok, emb_model, batch_size: int = None) -> np.ndarray:
    """テキストをベクトル化"""
    if emb_tok is None or emb_model is None:
        return None
    
    # 設定値を使用
    if batch_size is None:
        batch_size = cfg.batch_size
    
    vecs = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]
        inputs = emb_tok(batch, padding=True, truncation=True, return_tensors="pt").to(cfg.device)
        with torch.no_grad():
            outputs = emb_model(**inputs)
            emb = mean_pooling(outputs, inputs["attention_mask"])
            emb = torch.nn.functional.normalize(emb, p=2, dim=1)
            vecs.append(emb.cpu().numpy())
    return np.vstack(vecs)
    
def build_faiss_index(embs: np.ndarray):
    index = faiss.IndexFlatIP(embs.shape[1])
    index.add(embs)
    return index


def vector_search_top_k(query: str,
                        chunks: List[str],
                        emb_tok: AutoTokenizer,
                        emb_model: AutoModel,
                        index,
                        k: int) -> List[Tuple[int, float]]:
    import faiss  # noqa
    with torch.no_grad():
        q_inputs = emb_tok([query], padding=True, truncation=True, return_tensors="pt").to(cfg.device)
        q_out = emb_model(**q_inputs)
        q_vec = mean_pooling(q_out, q_inputs["attention_mask"])
        q_vec = torch.nn.functional.normalize(q_vec, p=2, dim=1).cpu().numpy()
    sims, ids = index.search(q_vec, k)
    return [(int(i), float(s)) for i, s in zip(ids[0], sims[0]) if 0 <= i < len(chunks)]






def _snap_to_sentence_boundary(text: str) -> str:
    m = re.search(r'(.+?[。．.!！?？])(?:[^。．.!！?？]*)$', text)
    return m.group(1) if m else text

def truncate_to_token_limit(tok: AutoTokenizer, text: str, limit: int) -> str:
    ids = tok(text, add_special_tokens=False).input_ids
    if len(ids) <= limit:
        return _snap_to_sentence_boundary(text)
    clipped = tok.decode(ids[:limit], skip_special_tokens=True)
    return _snap_to_sentence_boundary(clipped)

# =====
# 生成
# =====
@torch.no_grad()
def generate_with_limit(cfg,
                        prompt: str,
                        llm_tok: AutoTokenizer,
                        llm: AutoModelForCausalLM) -> str:
    inputs = llm_tok(prompt, return_tensors="pt", truncation=True, max_length=2048).to(cfg.device)
    out = llm.generate(
        **inputs,
        max_new_tokens=cfg.answer_token_limit + cfg.answer_headroom,
        temperature=cfg.temperature,
        top_p=0.9,
        do_sample=True,
        repetition_penalty=1.05,
        pad_token_id=llm_tok.pad_token_id,
        eos_token_id=llm_tok.eos_token_id or llm_tok.pad_token_id,
    )
    raw = llm_tok.decode(out[0], skip_special_tokens=True)
    
    # プロンプトを除去して回答部分のみを取得
    if "[回答]" in raw:
        # RAGモードの場合
        text = raw.split("[回答]", 1)[-1].strip()
    else:
        # RAGなしモードの場合：プロンプト部分を除去
        prompt_tokens = llm_tok(prompt, return_tensors="pt", add_special_tokens=False).input_ids[0]
        generated_tokens = out[0][len(prompt_tokens):]
        text = llm_tok.decode(generated_tokens, skip_special_tokens=True).strip()
    
    return truncate_to_token_limit(llm_tok, text, cfg.answer_token_limit)

# モデルの読み込み状態を確認する関数
def get_model_status():
    """現在のモデル読み込み状況を取得"""
    status = {
        "llm_loaded": llm_tok is not None and llm is not None,
        "emb_loaded": emb_tok is not None and emb_model is not None,
        "lora_loaded": llm_with_lora is not None
    }
    return status

def get_model_status_text():
    """モデル読み込み状況をテキストで取得"""
    status = get_model_status()
    
    status_text = "📊 **モデル読み込み状況**:\n"
    status_text += f"• 生成モデル: {'✅ 読み込み済み' if status['llm_loaded'] else '❌ 未読み込み'}\n"
    status_text += f"• 埋め込みモデル: {'✅ 読み込み済み' if status['emb_loaded'] else '❌ 未読み込み'}\n"
    status_text += f"• LoRAアダプター: {'✅ 読み込み済み' if status['lora_loaded'] else '❌ 未読み込み'}\n"
    
    return status_text

# 実行パス
def run_pipeline(question, rag_mode=""):
    start_time = time.time()
    
    # GPUメモリをクリア
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    print("推論処理を開始しています...")
    
    # グローバル変数から取得（事前読み込み済み）
    global llm_tok, llm, emb_tok, emb_model, llm_with_lora
    
    # モデルの読み込み状態を確認
    if llm_tok is None or llm is None:
        return "エラー: 生成モデルが読み込まれていません", 0.0

    # === モデル選択とモード判定 ===
    is_ft = (cfg.model_type == "finetuning" and llm_with_lora is not None)
    current_model = llm_with_lora if is_ft else llm
    use_rag = (rag_mode == "RAGあり")
    
    print(f"モード: {'FT' if is_ft else 'ベース'}モデル{' + RAG' if use_rag else 'のみ'}")

    # === RAG処理（必要な場合のみ） ===
    retrieved = None
    if use_rag:
        if emb_tok is None or emb_model is None:
            return "エラー: RAGモードには埋め込みモデルが必要です", time.time() - start_time
        if not update_rag_data():
            return "エラー: RAGデータの更新に失敗しました", time.time() - start_time

        first_stage = vector_search_top_k(question, chunk_data, emb_tok, emb_model, faiss_index, k=cfg.top_k_retrieve)
        cand_ids = [i for i, _ in first_stage]
        retrieved = [chunk_data[i] for i in cand_ids]

    # === プロンプト構築と推論 ===
    if use_rag:
        # RAGあり（ベース・FT共通でRAGプロンプトを使用）
        prompt = build_prompt_rag(cfg, question, retrieved, llm_tok)
        answer = generate_with_limit(cfg, prompt, llm_tok, current_model)
        answer += f"\n\n📚 **参考**:\n{retrieved[0]}"
    else:
        # RAGなし（ベース・FT共通でFTプロンプトを使用）
        prompt = build_prompt_ft(cfg, question, llm_tok)
        answer = generate_with_limit(cfg, prompt, llm_tok, current_model)

    print(f"処理時間: {time.time() - start_time:.2f}秒")
    return answer, time.time() - start_time

def load_clean_base_model():
    global llm_tok, llm
    
    try:
        model_name = cfg.llm_model_name
        print(f"オリジナルモデルを読み込み中: {model_name}")
        
        # トークナイザーを読み込み
        llm_tok = AutoTokenizer.from_pretrained(model_name)
        
        # 4bit量子化設定（メモリ最適化）
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16
        )
        
        # モデルを読み込み（シンプルに）
        llm = AutoModelForCausalLM.from_pretrained(
            model_name,
            quantization_config=bnb_config,
            device_map="auto",
            torch_dtype=torch.float16,
            trust_remote_code=True,
            low_cpu_mem_usage=True
        )
        
        # 基本的な設定
        if llm_tok.pad_token_id is None and llm_tok.eos_token_id is not None:
            llm_tok.pad_token = llm_tok.eos_token
        llm.config.pad_token_id = llm_tok.pad_token_id or llm_tok.eos_token_id
        
        # 推論モードに設定
        llm.eval()
        
        print("✅ オリジナルモデルの読み込み完了")
        return llm_tok, llm
        
    except Exception as e:
        print(f"オリジナルモデルの読み込みエラー: {e}")
        return None, None

# Gradioインターフェースを作成
with gr.Blocks(title="ChangeLLMチャットボット", theme=gr.themes.Soft(primary_hue="blue", secondary_hue="gray"), css="""
    .large-text textarea {
        font-size: 20px !important;
    }
""") as demo:
    gr.Markdown("# ChangeLLMチャットボット 🤖")
    gr.Markdown("LLMを切り替えられるチャットボットです。")
    
    with gr.Tab("チャット"):
        gr.Markdown("## 設定")
        
        with gr.Row():
            with gr.Column():
                rag_mode_radio = gr.Radio(
                    choices=["RAGあり", "RAGなし"],
                    value="RAGあり",
                    label="RAGモード",
                    info="RAGあり：RAGモード、RAGなし：LLMのみ"
                )
            
            with gr.Column():
                model_radio = gr.Radio(
                    choices=["original", "finetuning"],
                    value="original",
                    label="モデル選択",
                    info="original：デフォルトのsarashina2.2-3B-instruct-v0.1モデルを使用、finetuning：LoRAアダプターを適用（DEFAULT_LORA_ADAPTER_PATHで指定）"
                )
        
        # 現在のデータファイル名を表示
        gr.Markdown(f"**📁 現在のデータファイル**: {DATA_FILE_PATH}")
        
        gr.Markdown("## チャット")
        
        # チャット履歴を表示
        chatbot = gr.Chatbot(label="Chatbot", type="messages")
        
        # メッセージ入力
        msg = gr.Textbox(label="質問", placeholder="質問を入力してください")
        
        # 送信ボタン
        submit_btn = gr.Button("送信", variant="primary")
        
        def respond(message, history, rag_mode, model_mode):
            if message.strip() == "":
                return "", history
            # モデルタイプを更新
            cfg.model_type = model_mode
            response, response_time = run_pipeline(message, rag_mode)
            
            # 応答時間を含む回答を作成
            response_with_time = f"{response}\n\n⏱️ 応答時間: {response_time:.2f}秒"
            
            # type="messages"の場合、historyは辞書のリスト形式
            history.append({"role": "user", "content": message})
            history.append({"role": "assistant", "content": response_with_time})
            
            return "", history
        
        # ラジオボタン選択変更時のイベントハンドラー
        def on_model_change(model_type):
            if model_type == "original":
                cfg.model_type = "original"
            else:
                cfg.model_type = "finetuning"
        

        
        def on_rag_mode_change(rag_mode):
            if rag_mode == "RAGなし":
                cfg.gen_mode = "FT"
            else:
                cfg.gen_mode = "RAG"
            return rag_mode
        
        
        
        # イベントハンドラー
        submit_btn.click(
            fn=respond,
            inputs=[msg, chatbot, rag_mode_radio, model_radio],
            outputs=[msg, chatbot]
        )
        
        # Enterキーでも送信
        msg.submit(
            fn=respond,
            inputs=[msg, chatbot, rag_mode_radio, model_radio],
            outputs=[msg, chatbot]
        )
        
        # ラジオボタンの選択変更を監視
        model_radio.change(
            fn=on_model_change,
            inputs=[model_radio],
            outputs=[]
        )
        
        rag_mode_radio.change(
            fn=on_rag_mode_change,
            inputs=[rag_mode_radio],
            outputs=[rag_mode_radio]
        )
        


# アプリケーションを起動
if __name__ == "__main__":
    print("=== チャットボット起動 ===")
    
    # 1. 生成モデル（sarashina）を読み込み
    print("1/3: 生成モデル（sarashina）を読み込み中")
    print("   - オリジナルモデルを読み込み中")
    llm_tok, llm = load_clean_base_model()
    if llm_tok is None or llm is None:
        print("エラー: オリジナルモデルの読み込みに失敗しました")
        exit(1)
    print("✅ オリジナルモデルの読み込み完了")
    
    # 2. 埋め込みモデル（bge-m3）を読み込み
    print("2/3: 埋め込みモデルを読み込み中")
    try:
        emb_tok = AutoTokenizer.from_pretrained(cfg.emb_model_name)
        emb_model = AutoModel.from_pretrained(cfg.emb_model_name).to(cfg.device).eval()
        print("✅ 埋め込みモデルの読み込み完了")
    except Exception as e:
        print(f"エラー: 埋め込みモデルの読み込みに失敗しました: {e}")
        exit(1)
    
    # 3. LoRAアダプターを事前に適用
    print("3/3: LoRAアダプターを事前適用中...")
    if os.path.exists(DEFAULT_LORA_ADAPTER_PATH):
        try:
            llm_with_lora = apply_lora_adapter(llm, DEFAULT_LORA_ADAPTER_PATH)
            if llm_with_lora is not None:
                print("✅ LoRAアダプターの事前適用完了")
                print(f"   - ベースモデル: {cfg.llm_model_name}")
                print(f"   - LoRAアダプター: {DEFAULT_LORA_ADAPTER_PATH}")
            else:
                print("⚠️ LoRAアダプターの適用に失敗、ベースモデルのみで続行します")
                llm_with_lora = None
        except Exception as e:
            print(f"警告: LoRAアダプターの事前適用に失敗しました: {e}")
            print("オリジナルモデルのみで続行します")
            llm_with_lora = None
    else:
        print("LoRAアダプターパスが存在しないため、オリジナルモデルのみで続行します")
        llm_with_lora = None
    
    print("🎉 すべてのモデルの読み込みが完了しました！")
    print("UIを起動しています...")
    
    port = find_available_port(7880)
    if port is None:
        print("エラー: 利用可能なポートが見つかりませんでした。")
        exit(1)
    
    print(f"ポート {port} で起動します...")
    
    try:
        demo.launch(
            inbrowser=True,
            server_name="0.0.0.0",
            server_port=port,
            share=False,
            show_error=True
        )
    except OSError as e:
        if "Address already in use" in str(e):
            print(f"ポート {port} が使用中です。別のポートを試します...")
            # 別のポートで再試行
            port = find_available_port(port + 1)
            if port is None:
                print("エラー: 利用可能なポートが見つかりませんでした。")
                exit(1)
            
            print(f"ポート {port} で再起動します...")
            demo.launch(
                inbrowser=True,
                server_name="0.0.0.0",
                server_port=port,
                share=False,
                show_error=True
            )
        else:
            print(f"起動中にエラーが発生しました: {e}")
            raise

d:\conda_envs\ibaraki\Lib\site-packages\transformers\utils\hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


=== チャットボット起動 ===
1/3: 生成モデル（sarashina）を読み込み中
   - オリジナルモデルを読み込み中
オリジナルモデルを読み込み中: ./sbintuitions/sarashina2.2-3B-instruct-v0.1


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ オリジナルモデルの読み込み完了
✅ オリジナルモデルの読み込み完了
2/3: 埋め込みモデルを読み込み中
✅ 埋め込みモデルの読み込み完了
3/3: LoRAアダプターを事前適用中...
LoRAアダプターを適用中: ./output/checkpoint-160
LoRA設定を読み込みました: sbintuitions/sarashina2.2-3b-instruct-v0.1
LoRAアダプターの適用完了
✅ LoRAアダプターの事前適用完了
   - ベースモデル: ./sbintuitions/sarashina2.2-3B-instruct-v0.1
   - LoRAアダプター: ./output/checkpoint-160
🎉 すべてのモデルの読み込みが完了しました！
UIを起動しています...
ポート 7880 で起動します...
* Running on local URL:  http://0.0.0.0:7880
* To create a public link, set `share=True` in `launch()`.


推論処理を開始しています...
モード: ベースモデル + RAG
処理時間: 16.30秒
推論処理を開始しています...
モード: ベースモデルのみ
処理時間: 1.85秒
推論処理を開始しています...
モード: FTモデルのみ
処理時間: 1.97秒
推論処理を開始しています...
モード: ベースモデルのみ
処理時間: 7.95秒
推論処理を開始しています...
モード: FTモデルのみ
処理時間: 8.50秒
推論処理を開始しています...
モード: FTモデル + RAG
処理時間: 7.64秒
推論処理を開始しています...
モード: ベースモデル + RAG
処理時間: 12.45秒
